# Introduction to tf.keras

In [ ]:
import sys
import shutil
import tensorflow as tf

import numpy as np
%matplotlib inline
import random
import matplotlib.pyplot as plt

In [ ]:
# Python version 3.5 or 3.6
assert sys.version_info >= (3, 5)
assert sys.version_info < (3, 7)
# Tensorflow 2.0
assert tf.__version__ >= "2.0"

We will take a look at a first concrete example of a neural network, which makes use of `tf.keras` to learn to classify 
images. 

The problem we are trying to solve here is to classify RGB images (32 pixels by 32 pixels), into their 10 categories (_airplane_, _automobile_, _bird_, _cat_, _deer_, _dog_, _frog_, _horse_, _ship_, _truck_). The dataset we will use is the CIFAR10 dataset, a classic dataset in the machine learning community.

# Input Data Management

## Download the dataset

The CIFAR10 dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays.

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

## Visualize the data

Let's print the shapes of the datasets

In [ ]:
print("Train images shape : {}".format(train_images.shape))
print("Train labels shape : {}".format(train_labels.shape))
print("Test images shape : {}".format(test_images.shape))
print("Test labels shape : {}".format(test_labels.shape))

Let's see how the images look. This function shows a random example along with it's corresponding label.

In [ ]:
i = random.randint(0, 100)

print("Label: %s" % train_labels[i])
plt.imshow(train_images[i], cmap='gray')

Thats a little blurry !

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we will verify if these predictions match the labels from `test_labels`.

## Reformat the data

The labels have values from 0 to 9, but neural networks need instead to have access to a one-hot encoded vector (a vector of length 10 with all zeros but one at the index of the label).

> <div class="mark">Use the function to_categorical to perform one-hot encoding on the target variable</div><i class="fa fa-lightbulb-o "></i>

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical

In [ ]:
NUM_CAT = 10

train_cat  = tf.keras.utils.to_categorical(train_labels, NUM_CAT)
test_cat = tf.keras.utils.to_categorical(test_labels, NUM_CAT)

print("Before", train_labels[0]) # The format of the labels before conversion
print("After", train_cat[0]) # The format of the labels after conversion

assert(len(train_cat[0]) == 10)

In [ ]:
NUM_CAT = 10

train_cat  = # TODO
test_cat = # TODO

print("Before", train_labels[0]) # The format of the labels before conversion
print("After", train_cat[0]) # The format of the labels after conversion

assert(len(train_cat[0]) == 10)

Train and test images values are integers from 1 to 255. We need to convert then into floats with values from 0 to 1.

> <div class="mark">Convert the pixel values from integers between 0 and 255 to floats between 0 and 1</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
train_images_01 = train_images.astype(np.float32) / 255
test_images_01 = test_images.astype(np.float32) / 255

assert(train_images_01.dtype == "float32")
assert(np.max(train_images_01) == 1)
assert(np.min(train_images_01) == 0)

In [ ]:
train_images_01 = # TODO
test_images_01 = # TODO

assert(train_images_01.dtype == "float32")
assert(np.max(train_images_01) == 1)
assert(np.min(train_images_01) == 0)

# Model Management

The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a succession of increasingly refined data filters -- the "layers".

The last layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each score will be the probability that the current image belongs to one of our 10 classes.

## Build the model

Our Neural Network will contain the following building blocks : 
- `Conv2D` Layer : 16 filters, (3, 3) kernel, relu activation, input shape (32, 32, 3)
- `MaxPooling2D` : pool size (2, 2)
- `Conv2D` Layer : 32 filters, (3, 3) kernel, relu activation
- `MaxPooling2D` : pool size (2, 2)
- `Flatten` layer
- `Dense` Layer : 10 neurons, softmax activation

> <div class="mark">Build the model with the given building blocks</div><i class="fa fa-lightbulb-o "></i>

Hint : create the network by using the `Sequential` API of Keras

Documentation : 
- https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
model = tf.keras.models.Sequential()
# TODO

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly classified).

You will implement the following compilation step for your Neural Network : 
- "adam" optimizer
- "categorical_crossentropy" loss
- metric : "accuracy"

> <div class="mark">Compile the model</div><i class="fa fa-lightbulb-o "></i>

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile

In [ ]:
optimizer = tf.optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
optimizer = tf.optimizers.Adam()

model. # TODO

Summarize the model

In [ ]:
model.summary()

## Train the model

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

You will fit the network with the following configurations :
- `x`: The prepared images training set
- `y`: The prepared labels
- `epochs` : 5 (passes on the whole dataset)
- `batch_size`: 32 images
- `validation_date`: prepared images and labels for test set
- `callbacks`: tensorboard

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit

You will also add a callback for launching TensorBoard to observe how the training is performing.

In [ ]:
LOG_DIR = './tensorboard/tf_keras'

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1, update_freq="batch")

> <div class="mark">Fit the model with the above information.</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
shutil.rmtree(LOG_DIR, ignore_errors=True)

model.fit(x=train_images_01, 
         y=train_cat, 
         epochs=5, 
         batch_size=32, 
         validation_data=(test_images_01, test_cat),
         callbacks=[tensorboard])

In [ ]:
shutil.rmtree(LOG_DIR, ignore_errors=True)

model. # TODO

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over the training data.

## Performance visualisation with Tensorboard

After executing the following command, click on the suggested link (virtualenv users) or open a new tab and go to 127.0.0.1:6006 (Docker users) to open Tensorboard.

Once finished, click on the STOP botton.

In [ ]:
!tensorboard --logdir "tensorboard/tf_keras" --port 6006

# Model Performance Evaluation

Now let's check that our model performs well on the test set too.

You can do this by calling the `evaluate` method of your network.

Documentation : https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate

> <div class="mark">Evaluate the model performance on test set</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
loss, accuracy = model.evaluate(test_images_01, test_cat)

print('Test accuracy: %.2f' % (accuracy))

In [ ]:
loss, accuracy = model. # TODO

print('Test accuracy: %.2f' % (accuracy))